# Introduction

Is there any relation between the suicide, as a social phenomenona, and the economic, criminal and gender diversity of a society?

Along this work, we will try to answer this question for the context of an specific country which provides us with access to series of datasets that are useful for our goal.

In the Argentina's government [public official datasets site](https://datos.gob.ar/dataset) we can find for our interest the following ones:

\underline{Principal Sources}

*   [Hechos y víctimas de suicidios en la República Argentina. Total nacional (panel)](https://datos.gob.ar/dataset/seguridad-suicidios-sistema-alerta-temprana-estadisticas-criminales-republica-argentina/archivo/seguridad_12.2): which recopiles the registered suicides in all the country since the year 2017 by the Sistema de Alerta Temprana del Sistema Nacional de Información Criminal.
*   [Estadísticas criminales en la República Argentina por provincias](https://datos.gob.ar/dataset/seguridad-snic---provincial-estadisticas-criminales-republica-argentina-por-provincias/archivo/seguridad_2.2): that give us a bunch of statistcs measures taken for each province of the country since the year 2000.
*   [Salario promedio por departamento/partido y clae2, total empresas](https://datos.gob.ar/dataset/produccion-salarios-por-departamentopartido-sector-actividad/archivo/produccion_76593966-9526-4a9d-b56c-87c8a4a20dd8): the mean wage for each department of the country and sector based on the AFIP's CLAE2 standard.

\underline{Secondary Sources}

*   [Diccionario de departamentos](https://datos.gob.ar/dataset/produccion-salarios-por-departamentopartido-sector-actividad/archivo/produccion_e8cb322b-6934-46b9-ae4e-713470287240): which provides the INDEC's standard names and ids for the departments of the country.
*   [Diccionario de claes](https://datos.gob.ar/dataset/produccion-salarios-por-departamentopartido-sector-actividad/archivo/produccion_8c7e4f21-750e-4298-93d1-55fe776ed6d4): that contains AFIP's categories to classify the economic activities of Argentina.

Which were mapped in `data_cleaning.ipynb` to a number of tables that follow the ERD:

![ERD](https://drive.google.com/uc?id=1islpEu6Dd5pdxKj9CztsnAgzJIQvN42F)

Givin us datasets for:

- Provinces
- Departments
- Crimes
- Suicides
- Mean Income
- CLAE

that are consistent between each other, preserve almost all the original data rows and drops out the variables that we not consider useful appropiate for this work.

With thst present, we can start.


# Data Analysis

## Used libraries

In [1]:
# !pip install inline_sql

In [2]:
# in case of using colab
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from inline_sql import sql

## Loading the datasets

In [4]:
working_dir = "/content/MyDrive/MyDrive/Proyectos/Suicidios_Análisis/"

In [16]:
provinces = pd.read_csv(working_dir+"Clean_Data/provincias.csv", sep=",")

In [17]:
departments = pd.read_csv(working_dir+"Clean_Data/departamentos.csv", sep=",")

In [18]:
crimes = pd.read_csv(working_dir+"Clean_Data/crimenes.csv", sep=",")

In [ ]:
suicides = pd.read_csv(working_dir+"Clean_Data/suicidios.csv", sep=",")

In [ ]:
mean_income = pd.read_csv(working_dir+"Clean_Data/ingresos_promedio.csv", sep=",")

In [ ]:
clae2 = pd.read_csv(working_dir+"Clean_Data/clae2.csv", sep=",")

In [15]:
suicides

,Unnamed: 0,id_hecho,id_departamento_indec,fecha,hora,lugar,modalidad,origen_registro,suicida_sexo,suicida_rango_edad,suicida_identidad_genero
0,2,8569,78021,01-01-2017,19:00:00,Vía pública,Ahorcamiento,Denuncia particular,Masculino,15-19,Sin determinar
1,3,8705,58035,31-01-2017,17:30:00,Otro Lugar ( Especificar ),Arma de fuego,Intervención policial,Masculino,20-24,Sin determinar
2,4,8706,58035,31-01-2017,22:40:00,Domicilio particular,Ahorcamiento,Intervención policial,Masculino,20-24,Sin determinar
3,5,8707,58035,29-01-2017,20:30:00,Domicilio particular,Ahorcamiento,Intervención policial,Masculino,20-24,Sin determinar
4,6,8708,58049,02-01-2017,14:25:00,Domicilio particular,Ahorcamiento,Intervención policial,Masculino,45-49,Sin determinar
...,...,...,...,...,...,...,...,...,...,...,...
21421,21713,38831,14021,28-08-2022,11:11:11,Domicilio particular,Ahorcamiento,Intervención policial,Femenino,15-19,Mujer
21422,21714,37640,18049,09-07-2022,07:30:00,Cárcel o comisaría,Ahorcamiento,Intervención policial,Masculino,30-34,Varón
21423,21715,37642,18070,11-09-2022,05:00:00,Domicilio particular,Sin determinar,Intervención policial,Masculino,50-54,Varón
21424,21716,37643,18070,27-09-2022,06:45:00,Domicilio particular,Ahorcamiento,Intervención policial,Masculino,15-19,Varón
